In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import os
import yaml
import torch
import logging
from typing import List, Dict, Any

import pandas as pd
from datasets import load_dataset, Dataset
from transformers import pipeline
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import evaluate
import re

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
bertscore = evaluate.load('bertscore')

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [61]:
def load_model_and_tokenizer(path):
    model = AutoModelForCausalLM.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
    ).to("cuda")

    model.eval()

    tokenizer = AutoTokenizer.from_pretrained(path) #"tiiuae/Falcon3-1B-Base") #path)
    
    return model, tokenizer

def predict_summary(test_input, model, tokenizer, max_length, max_new_tokens, model_name):

    # custom instruct prompt start
    prompt_template_test = f"Summarize this arabic text:\n{{text}}\n---\nSummary:\n"

    test_prompt = prompt_template_test.format(text=test_input)
    # print(test_prompt)
    
    # Tokenize and generate
    inputs = tokenizer(test_prompt, return_tensors="pt", max_length=max_length).to("cuda")
    outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            # num_beams=4,
            # early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,  # Crucial for stopping
        )


    # Decode the output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_summary = summary.split("Summary:")[-1].strip()
    # print('-'*100)
    # print(f"{summary}")
    # print('-'*100)
    # print(f"{cleaned_summary}")
    
    return {f"summary_{model_name}": cleaned_summary}

# Post-process text for ROUGE and BLEU
def postprocess_text(texts):
    # Remove punctuation using regex
    texts = [re.sub(r'[^\w\s]', '', t) for t in texts]
    
    # Tokenize into sentences using RegexpTokenizer
    sent_tokenizer = RegexpTokenizer(r'\w+')
    texts = [" ".join(sent_tokenizer.tokenize(t)) for t in texts]
    return texts

def compute_metrics(
        dataset: Dataset, 
        models: List[str], 
        reference_column: str = "summary"
    ) -> pd.DataFrame:
        """Compute comprehensive metrics"""
        results = []
        
        for model_path in models:
            model_name = model_path.split('/')[-1]
            summary_column = f"summary_{model_name}"
            
            # Extract predictions and references
            predictions = dataset[summary_column]
            references = dataset[reference_column]
            
            # Apply post-processing to predictions and labels
            predictions = postprocess_text(predictions)
            references = postprocess_text(references)
            
            print(f'predictions: {predictions[0]}')
            print(f'references: {references[0]}')
            print('-'*50)
            # print(f'predictions: {predictions[5]}')
            # print(f'references: {references[5]}')
            # print('-'*50)
            
            # Compute ROUGE
            rouge_results = rouge.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute BLEU
            bleu_results = bleu.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute METEOR
            meteor_results = meteor.compute(
                predictions=predictions, 
                references=references
            )
            
            # Compute BERTScore
            bertscore_results = bertscore.compute(
                predictions=predictions, 
                references=references, 
                lang='ar'
            )
            
            # Aggregate results
            model_metrics = {
                "model": model_name,
                "rouge1": rouge_results["rouge1"] * 100,
                "rouge2": rouge_results["rouge2"] * 100,
                "rougeL": rouge_results["rougeL"] * 100,
                "rougeLsum": rouge_results["rougeLsum"] * 100,
                "bleu": bleu_results["bleu"] * 100,
                "meteor": meteor_results["meteor"] * 100,
                "bertscore_precision": sum(bertscore_results['precision']) / len(bertscore_results['precision']) * 100,
                "bertscore_recall": sum(bertscore_results['recall']) / len(bertscore_results['recall']) * 100,
                "bertscore_f1": sum(bertscore_results['f1']) / len(bertscore_results['f1']) * 100
            }
            
            results.append(model_metrics)
        
        return pd.DataFrame(results)
    

NameError: name 'Dataset' is not defined

In [25]:
with open('eval_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [26]:
config

{'EVALUATE_MODELS': ['BounharAbdelaziz/Qwen2.5-0.5B-Instruct-bs-2-lr-1e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2'],
 'batch_size': 32,
 'MAX_LEN': 2048,
 'MODEL_PATH': 'BounharAbdelaziz/Falcon3-1B-Base-bs-2-lr-2e-05-ep-4-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-1024-v2',
 'DATASET_PATH': 'BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered',
 'TEXT_COLUMN': 'text',
 'REFERENCE_COLUMN': 'summary',
 'OUTPUT_PATH': 'metrics.csv',
 'SEED': 42}

In [27]:
models = config['EVALUATE_MODELS']
model, tokenizer = load_model_and_tokenizer(models[0])
model_name = models[0].split('/')[-1]
max_length = config['MAX_LEN']
max_new_tokens = max_length

In [28]:
dataset = load_dataset(config['DATASET_PATH'], split="test")
dataset = dataset.select(range(10))

In [29]:
dataset

Dataset({
    features: ['text', 'summary', 'summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length'],
    num_rows: 10
})

In [30]:
dataset = dataset.map(lambda row: predict_summary(row['text'], model, tokenizer, max_length, max_new_tokens, model_name))

Map: 100%|██████████| 10/10 [02:21<00:00, 14.14s/ examples]


In [31]:
metrics = compute_metrics(dataset, models, reference_column= "summary")

predictions: في القرن 1061 ميلادي كانت صقلية جزءا من منطقة شبه عربية لكنها تحولت إلى جزيرة عربية مملوكة لدول مختلفة خلال القرن 12 اندمجت الثقافة العربية مع الثقافة اليونانية مما أسفر عن تطور اللغة العربية في بلدة صقلية كما انتشرت الثقافة العربية في بلدة صقلية حيث كانت تعزز دورها في الإعلان عن الثقافة الإسلامية
references: في 1061 كانت صقلية مجزأة إلى خمس إمارات مع تنافس عربي وأمازيغي استطاع الملك النورماني روeger الأول السيطرة عليها وأصبحت باليرمو عاصمتها عام 1072 رغم فقدان العرب للسلطة السياسية ظلوا الثقافة الرئيسية وازدهرت فيها الأدب والعلم حتى القرن الثاني عشر بعد وفاة روeger الأول استمر هذا الوضع مع ابنه روجر الثاني
--------------------------------------------------


In [32]:
metrics

,model,rouge1,rouge2,rougeL,rougeLsum,bleu,meteor,bertscore_precision,bertscore_recall,bertscore_f1
0,Qwen2.5-0.5B-Instruct-bs-2-lr-1e-05-ep-1-wp-10...,3.40796,0.0,3.40796,3.40796,0.987424,12.953025,70.891491,72.063261,71.333671


In [ ]:
dataset[0]

In [ ]:
s = predict_summary(dataset[0]['text'], model, tokenizer, max_length, max_new_tokens, model_name)

In [ ]:
s

In [ ]:
models = [model_path]
metrics = compute_metrics(dataset, models, reference_column= "summary")

In [ ]:
metrics = compute_metrics(dataset, models, reference_column= "summary")

In [12]:
predict_summary(dataset[0]["text"], model, tokenizer, max_length, max_new_tokens, model_name)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


----------------------------------------------------------------------------------------------------
Summarize this arabic text:
في العام 1061 ميلادية كانت صقلية لا تزال جزيرة شبه عربية، لكنها كانت مجزأة إلى
خمس إمارات منقسمة على نفسها، وإضافة لذلك تسبب التنازع بين الأمراء، والتنافس
بين العرب والأمازيغ في الجزيرة الواقعة جنوبي إيطاليا، في تمكّن ملك النورمان
روجر الأول من إخضاع أغلب الجزيرة لحكمه، وبحلول العام 1072 أصبحت العاصمة
الصقلية باليرمو تحت سيطرة الرومان سيطرة كاملة. لكن أفول نجم العرب السياسي عن
صقلية لم يغيّب الثقافة العربية عن الجزيرة المتوسطية، وظل الوجود العربي قويا
فيها تحت حكم ملوك النورمان في هذه الفترة، وظل بلاط ملوك النورمان طوال القرن
الـ12 زاخرا بالعديد من الشعراء والعلماء والحرفيين العرب، الذين تولوا مناصب
إدارية رفيعة أيضا. وبعد وفاة روجر الأول عام 1101م خلفه ابنه روجر الثاني الذي
كان بلاطه يعج بالمثقفين العرب، ورفض الملك الصقلي المشاركة في الحروب الصليبية
رغم إلحاح البابا، وشاع استخدام اللغة العربية في إدارته المالية واحتفالاته
الفخمة وفي المجالس الأدبية والشعرية 

{'summary_Qwen2.5-0.5B-Instruct-bs-2-lr-1e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-2048-v2': 'في القرن 1061، كان صقلية جزءًا من إمارة النورمان. تطورت الثقافة العربية فيها خلال القرن 12، حيث ظهر الشعراء العرب في بلاطهم. وتحديداً، كان ملك النورمان روجر الثاني، الذي اشتهر بمجالسه العلمية والشعرية. كما كانت صقلية مركزاً لتبادل الثقافات بين المسلمين والإسلاميين، مما أثر على تشكيلة الشعراء العرب في تلك الفترة.'}

In [11]:
# custom instruct prompt start
prompt_template_test = f"Summarize this arabic text:\n{{text}}\n---\nSummary:\n"

test_prompt = prompt_template_test.format(text=dataset[0]["text"])
# print(test_prompt)

# Tokenize and generate
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        # num_beams=4,
        # early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,  # Crucial for stopping
    )


# Decode the output
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
cleaned_summary = summary.split("Summary:")[-1].strip()
print('-'*100)
print(f"{summary}")
print('-'*100)
print(f"{cleaned_summary}")

----------------------------------------------------------------------------------------------------
Summarize this arabic text:
في العام 1061 ميلادية كانت صقلية لا تزال جزيرة شبه عربية، لكنها كانت مجزأة إلى
خمس إمارات منقسمة على نفسها، وإضافة لذلك تسبب التنازع بين الأمراء، والتنافس
بين العرب والأمازيغ في الجزيرة الواقعة جنوبي إيطاليا، في تمكّن ملك النورمان
روجر الأول من إخضاع أغلب الجزيرة لحكمه، وبحلول العام 1072 أصبحت العاصمة
الصقلية باليرمو تحت سيطرة الرومان سيطرة كاملة. لكن أفول نجم العرب السياسي عن
صقلية لم يغيّب الثقافة العربية عن الجزيرة المتوسطية، وظل الوجود العربي قويا
فيها تحت حكم ملوك النورمان في هذه الفترة، وظل بلاط ملوك النورمان طوال القرن
الـ12 زاخرا بالعديد من الشعراء والعلماء والحرفيين العرب، الذين تولوا مناصب
إدارية رفيعة أيضا. وبعد وفاة روجر الأول عام 1101م خلفه ابنه روجر الثاني الذي
كان بلاطه يعج بالمثقفين العرب، ورفض الملك الصقلي المشاركة في الحروب الصليبية
رغم إلحاح البابا، وشاع استخدام اللغة العربية في إدارته المالية واحتفالاته
الفخمة وفي المجالس الأدبية والشعرية 

In [ ]:
model_path

# SFTTrainer models

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
from datasets import load_dataset, Dataset
import pandas as pd
import os
from tqdm import tqdm
from evaluate import load

# Set CUDA device
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Load metrics
rouge = load('rouge')
bleu = load('bleu')
meteor = load('meteor')
bertscore = load('bertscore')

# Batch size for inference
BATCH_SIZE = 128 // 2

# @torch.no_grad()
# def compute_metrics_causal_lm(eval_pred, tokenizer):
#     """Compute ROUGE and BLEU scores for evaluation."""
#     predictions, references = eval_pred

#     # Clip token IDs to the valid range
#     vocab_size = tokenizer.vocab_size

#     def clip_token_ids(token_ids):
#         """Clip token IDs to the valid range [0, vocab_size - 1]."""
#         return [min(max(token_id, 0), vocab_size - 1) for token_id in token_ids]

#     # Decode predictions and references in batches
#     decoded_preds = tokenizer.batch_decode([clip_token_ids(pred) for pred in predictions], skip_special_tokens=True)
#     decoded_refs = tokenizer.batch_decode([clip_token_ids(ref) for ref in references], skip_special_tokens=True)

#     # Compute ROUGE and BLEU scores
#     rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_refs, use_stemmer=True)
#     bleu_results = bleu.compute(predictions=decoded_preds, references=decoded_refs)

#     metrics = {key: rouge_results[key] * 100 for key in ["rouge1", "rouge2", "rougeL", "rougeLsum"]}
#     metrics["bleu"] = bleu_results["bleu"] * 100

#     return metrics
@torch.no_grad()
def compute_metrics_causal_lm(eval_pred, tokenizer):
    """Compute ROUGE and BLEU scores for evaluation."""
    predictions, references = eval_pred

    # Clip token IDs to the valid range
    vocab_size = tokenizer.vocab_size

    def clip_token_ids(token_ids):
        """Clip token IDs to the valid range [0, vocab_size - 1]."""
        return [min(max(token_id, 0), vocab_size - 1) for token_id in token_ids]

    # Decode predictions and references
    decoded_preds = [
        tokenizer.decode(clip_token_ids(pred), skip_special_tokens=True)
        for pred in predictions
    ]
    decoded_refs = [
        tokenizer.decode(clip_token_ids(ref), skip_special_tokens=True)
        for ref in references
    ]
    
    # Clean summaries
    def clean_summary(text):
        special_tokens = ["<|im_end|>", "<|assistant|>", "<|user|>", "<|system|>"]
        for token in special_tokens:
            text = text.replace(token, "")
        return re.sub(r"\s+", " ", text).strip()
    
    pred_summaries = []
    for pred in decoded_preds:
        if "<|assistant|>" in pred:
            summary = pred.split("<|assistant|>")[-1].strip()
            summary = clean_summary(summary)
            pred_summaries.append(summary)
        else:
            summary = pred.strip()
            summary = clean_summary(summary)
            pred_summaries.append(summary)
            
    # apply the same to the references
    ref_summaries = []
    for ref in decoded_refs:
        if "<|assistant|>" in ref:
            summary = ref.split("<|assistant|>")[-1].strip()
            summary = clean_summary(summary)
            ref_summaries.append(summary)
        else:
            summary = ref.strip()
            summary = clean_summary(summary)
            ref_summaries.append(summary)
            
    # print(f'0 - ref_summaries[0]: {ref_summaries[0]}')
    
    # Convert to token IDs
    pred_token_ids = [tokenizer.encode(p, add_special_tokens=False) for p in pred_summaries]
    ref_token_ids = [tokenizer.encode(r, add_special_tokens=False) for r in ref_summaries]

    # Use the exact same metric function from training
    eval_pred = (pred_token_ids, ref_token_ids)
    
    predictions, references = eval_pred

    # Clip token IDs to the valid range
    vocab_size = tokenizer.vocab_size

    # Decode predictions and references in batches
    decoded_preds = tokenizer.batch_decode([clip_token_ids(pred) for pred in predictions], skip_special_tokens=True)
    decoded_refs = tokenizer.batch_decode([clip_token_ids(ref) for ref in references], skip_special_tokens=True)
    
    # Print decoded examples to inspect issues
    print(f'decoded_preds[0]: {decoded_preds[0]}')
    print(f'decoded_refs[0]: {decoded_refs[0]}')

    # Compute ROUGE and BLEU scores
    rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_refs, use_stemmer=True)
    bleu_results = bleu.compute(predictions=decoded_preds, references=decoded_refs)

    metrics = {key: rouge_results[key] * 100 for key in ["rouge1", "rouge2", "rougeL", "rougeLsum"]}
    metrics["bleu"] = bleu_results["bleu"] * 100

    return metrics

# def summarize_batch(inputs, model, tokenizer, max_new_tokens=256, device="cuda"):
#     """Summarize a batch of texts using the trained model."""
#     model.eval()
#     model.to(device)

#     # # Apply chat template to all texts in the batch
#     # formatted_inputs = [tokenizer.apply_chat_template([{"role": "user", "content": text}], tokenize=False) for text in texts]

#     # # Tokenize the batch
#     # inputs = tokenizer(formatted_inputs, return_tensors="pt", padding=True, truncation=True).to(device)

#     generation_config = model.generation_config
#     # print(f'generation_config: {generation_config}')
    
#     # Generate summaries
#     with torch.no_grad():
#         outputs = model.generate(
#             input_ids=torch.LongTensor(inputs['input_ids']).to(device=device),
#             attention_mask=torch.LongTensor(inputs['attention_mask']).to(device=device),
#             max_new_tokens=max_new_tokens,            # Adjust based on input length
#             # temperature=0.5,               # Balanced determinism
#             # top_k=50,                      # Focus on top 50 tokens
#             # top_p=0.9,                     # Nucleus sampling threshold
#             # num_beams=4,                   # Beam search for coherence
#             # repetition_penalty=1.3,        # Reduce redundancy
#             # length_penalty=1.0,            # Neutral length control
#             # no_repeat_ngram_size=3,        # Avoid repeating 3-grams
#             # early_stopping=True,           # Stop when beams converge
#             # Inherit other configs from generation_config:
#             bos_token_id=generation_config.bos_token_id,
#             eos_token_id=generation_config.eos_token_id,
#             pad_token_id=generation_config.pad_token_id,
#             # num_beams=4,
#             # early_stopping=True,
#             # do_sample=False
#             # temperature=generation_config.temperature,
#             # top_k=generation_config.top_k,
#             # top_p=generation_config.top_p,
#             # do_sample=generation_config.do_sample,
#             # repetition_penalty=generation_config.repetition_penalty,
#         )
        

#     # Decode the generated outputs
#     generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     # Extract summaries (text after <|assistant|>)
#     summaries = []
#     for text in generated_texts:
#         if "<|assistant|>" in text:
#             summaries.append(text.split("<|assistant|>")[-1].strip())
#         else:
#             summaries.append(text.strip())

#     # Clean summaries
#     def clean_summary(text):
#         special_tokens = ["<|im_end|>", "<|assistant|>", "<|user|>", "<|system|>"]
#         for token in special_tokens:
#             text = text.replace(token, "")
#         return re.sub(r"\s+", " ", text).strip()

#     return [clean_summary(summary) for summary in summaries]

# def summarize_dataset(dataset, model, tokenizer, model_name, max_new_tokens=256, device="cuda"):
#     """Summarize all texts in the dataset using the trained model."""

#     # Summarize in batches
#     summaries = []
#     for i in tqdm(range(0, len(dataset), BATCH_SIZE)):
#         # batch = dataset[i:i + BATCH_SIZE]['text']
#         batch = dataset[i:i + BATCH_SIZE]
#         batch_summaries = summarize_batch(batch, model, tokenizer, max_new_tokens, device)
#         summaries.extend(batch_summaries)

#     # Add summaries to the dataset
#     dataset = dataset.add_column(f"summary_{model_name}", summaries)
#     return dataset


import torch
from tqdm import tqdm
import re

def summarize_batch(inputs, model, tokenizer, max_new_tokens=256, device="cuda"):
    """Summarize a batch of texts using the trained model."""
    model.eval()
    model.to(device)

    generation_config = model.generation_config

    # Generate summaries
    with torch.no_grad():
        outputs = model.generate(
            input_ids=torch.LongTensor(inputs['input_ids']).to(device=device),
            attention_mask=torch.LongTensor(inputs['attention_mask']).to(device=device),
            max_new_tokens=max_new_tokens,
            bos_token_id=generation_config.bos_token_id,
            eos_token_id=generation_config.eos_token_id,
            pad_token_id=generation_config.pad_token_id,
        )

    # Decode the generated outputs
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract summaries (text after <|assistant|>)
    summaries = []
    for text in generated_texts:
        if "<|assistant|>" in text:
            summaries.append(text.split("<|assistant|>")[-1].strip())
        else:
            summaries.append(text.strip())

    # Clean summaries
    def clean_summary(text):
        special_tokens = ["<|im_end|>", "<|assistant|>", "<|user|>", "<|system|>"]
        for token in special_tokens:
            text = text.replace(token, "")
        return re.sub(r"\s+", " ", text).strip()

    return [clean_summary(summary) for summary in summaries]

def summarize_dataset(dataset, model, tokenizer, model_name, max_new_tokens=256, device="cuda", batch_size=8):
    """Summarize all texts in the dataset using the trained model."""
    model.eval()
    model.to(device)

    # Summarize in batches
    summaries = []
    for i in tqdm(range(0, len(dataset), batch_size)):
        # Get a batch of examples
        batch = dataset[i:i + batch_size]
        
        print(f'batch: {batch}')
        print(f'batch[text]: {batch['text']}')
        
        # Prepare the messages for the model using the tokenizer's chat template
        messages = [
            [{"role": "user", "content": example['text']}] for example in batch
        ]
        
        # Tokenize the batch
        inputs = tokenizer.apply_chat_template(
            messages, 
            truncation=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(device)
        
        # Summarize the batch
        batch_summaries = summarize_batch(inputs, model, tokenizer, max_new_tokens, device)
        summaries.extend(batch_summaries)

    # Add summaries to the dataset
    dataset = dataset.add_column(f"summary_{model_name}", summaries)
    return dataset

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def create_conversation(example):
    """
    Transform the dataset into a conversational format.
    The user provides the text, and the assistant provides the summary.
    """
    # Create a conversation with user and assistant roles
    messages = [
        {"role": "user", "content": example["text"]},  # User provides the text
    ]
    # Return the conversation as a dictionary
    return {"messages": messages}

def apply_chat_template(example, tokenizer):
    """ Apply the chat template to the dataset. """
    example["text"] = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    return example

def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, return_tensors="pt")

In [60]:
import torch
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

# model_path = "BounharAbdelaziz/Falcon3-1B-Base-bs-1-lr-1e-05-ep-1-wp-100-gacc-32-gnm-1.0-FP16-SFT-mx-1024-v2" 
# model_path = "BounharAbdelaziz/Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v2"
model_path = "BounharAbdelaziz/Qwen2.5-0.5B-bs-2-lr-0.0001-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v5"

# torch_dtype = torch.float32
torch_dtype = torch.float16

MAX_LEN=2048

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch_dtype).to("cuda")
model.use_cache = True
tokenizer = AutoTokenizer.from_pretrained(model_path)
model_name = model_path.split('/')[-1].strip()

tokenizer.padding_side='left'

# Set reasonable default for models without max length
tokenizer.model_max_length = MAX_LEN

# Set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
# Set chat template
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

# Load dataset
eval_dataset = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered", split='validation')
# eval_dataset = eval_dataset.select(range(10))

In [61]:
def summarize(text, model_name):
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    messages = [
        {"role": "user", "content": text},
    ]

    # prepare the messages for the model
    input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

    # inference
    outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=256,
            # do_sample=True,
            # temperature=0.7,
            # top_k=50,
            # top_p=0.95
    )
    summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    # print('raw: ', summary)
    
    if "<|assistant|>" in summary:
        summary = summary.split("<|assistant|>")[-1].strip()
    else:
        summary = summary.strip()

    # Clean summaries
    def clean_summary(text):
        special_tokens = ["<|im_end|>", "<|assistant|>", "<|user|>", "<|system|>"]
        for token in special_tokens:
            text = text.replace(token, "")
        return re.sub(r"\s+", " ", text).strip()

    summary = clean_summary(summary)
    # print('cleaned: ', summary)
    
    
    return {f'summary_{model_name}': summary}

In [62]:
eval_dataset = eval_dataset.remove_columns([f"summary_{model_path.split('/')[-1]}"])

ValueError: Column name ['summary_Qwen2.5-0.5B-bs-2-lr-0.0001-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v5'] not in the dataset. Current columns in the dataset: ['text', 'summary', 'summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length']

In [55]:
eval_dataset

Dataset({
    features: ['text', 'summary', 'summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length'],
    num_rows: 10
})

In [56]:
summary = summarize(eval_dataset[0]['text'], model_name)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [57]:
summary

{'summary_Qwen2.5-0.5B-bs-2-lr-0.0001-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v5': 'تعد الجنس من أفضل الطرق لبناء الثقة والتقارب، لكن يجب التفكير في الظروف المحيطة. يمكن لمشاهدة فيلم أو الاستماع إلى أغاني رائعة أن يساعدان على تخفيف التوتر. إذا كنت قلقة من التجاوز، ابتعد عن شريكك واحترمي رغباتك. إذا كنت تشعرين بالسوء حيال الاستمرار، ابتعد عن ذلك. إذا حاول شريكك إقناعك بتجاوز الحدود، احترمي رغباتك واحترمي رأيك. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك. إذا كنت تشعرين بالسوء حيال الاستمرار، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا.'}

In [63]:
eval_dataset = eval_dataset.map(lambda row: summarize(row['text'], model_name))

Map:   0%|          | 0/437 [00:00<?, ? examples/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Map:   0%|          | 1/437 [00:05<42:16,  5.82s/ examples]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Map:   0%|          | 2/437 [00:11<41:44,  5.76s/ examples]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Map:   1%|          | 3/437 [00:17<41:39,  5.76s/ examples]The attention ma

In [65]:
predictions = eval_dataset[f"summary_{model_path.split('/')[-1]}"]
references = eval_dataset["summary"]

# Convert to token IDs like during training
pred_token_ids = [tokenizer.encode(p) for p in predictions]
ref_token_ids = [tokenizer.encode(r) for r in references]

# Use the exact same metric function from training
eval_pred = (pred_token_ids, ref_token_ids)

metrics = compute_metrics_causal_lm(eval_pred, tokenizer)
print(metrics)

decoded_preds[0]: تعد الجنس من أفضل الطرق لبناء الثقة والتقارب، لكن يجب التفكير في الظروف المحيطة. يمكن لمشاهدة فيلم أو الاستماع إلى أغاني رائعة أن يساعدان على تخفيف التوتر. إذا كنت قلقة من التجاوز، ابتعد عن شريكك واحترمي رغباتك. إذا كنت تشعرين بالسوء حيال الاستمرار، ابتعد عن ذلك. إذا حاول شريكك إقناعك بتجاوز الحدود، احترمي رغباتك واحترمي رأيك. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك. إذا كنت تشعرين بالسوء حيال الاستمرار، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا. إذا كنت تشعرين بالسوء حيال قرارك، ابتعد عن ذلك أيضا.
decoded_refs[0]: في العلاقات السليمة، يمكنك بناء الثقة والتقارب من خلال التحدث والاستماع والمشاركة واحترام أفكاركما. يمكن استخدام نشاطات غير جنسية مثل اللعب بألعاب لوحية أو القراءة معا. يمكنك أيضا إجراء استطلاعات رأي أو مشاهدة فيلم مع وسائد وأضواء. إذا شعرت بالقلق، ابتعدي قليلا أو تقدمي لسببًا لنقل المكان. يجب أن يحترم شريكك رغباتك وي

In [13]:
eval_dataset

Dataset({
    features: ['text', 'summary', 'summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'summary_Qwen2.5-0.5B-bs-2-lr-0.0001-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v5'],
    num_rows: 10
})

In [14]:
# Transform the dataset into a conversational format
# eval_dataset = eval_dataset.map(create_conversation, remove_columns=["text", 'summary_model_name', 'tokenizer_name', 'dataset_source'])

# eval_dataset = eval_dataset.map(
#     apply_chat_template,
#     num_proc=os.cpu_count(),
#     fn_kwargs={"tokenizer": tokenizer},
#     remove_columns=["messages"],
#     desc="Applying chat template..."
# )

# eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Summarize dataset
eval_dataset = summarize_dataset(eval_dataset, model, tokenizer, model_name, max_new_tokens=256, device="cuda")

predictions = eval_dataset[f"summary_{model_path.split('/')[-1]}"]
references = eval_dataset["summary"]

# Convert to token IDs like during training
pred_token_ids = [tokenizer.encode(p) for p in predictions]
ref_token_ids = [tokenizer.encode(r) for r in references]

# Use the exact same metric function from training
eval_pred = (pred_token_ids, ref_token_ids)

metrics = compute_metrics_causal_lm(eval_pred, tokenizer)
print(metrics)

  0%|          | 0/2 [00:00<?, ?it/s]


batch: {'text': ['يوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك.  إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قراءة بعض القصص الرومانسية لبعضكما من أجل تشتيت الانتباه بعيدا عن الجنس (وربما كي تضحكا على مدى رداءة القصص). اجلسا معا واستمعا إلى ألبوم. ابحثي عن كلمات الأغاني واجلسي بالقرب من شريكك وتمتعي بقرب ج

TypeError: string indices must be integers, not 'str'

In [21]:
eval_dataset

Dataset({
    features: ['summary', 'sequence_length', 'text', 'input_ids', 'attention_mask', 'summary_Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v2'],
    num_rows: 50
})

In [ ]:
eval_dataset[0]

{'summary': 'في العلاقات السليمة، يمكنك بناء الثقة والتقارب من خلال التحدث والاستماع والمشاركة واحترام أفكاركما. يمكن استخدام نشاطات غير جنسية مثل اللعب بألعاب لوحية أو القراءة معا. يمكنك أيضا إجراء استطلاعات رأي أو مشاهدة فيلم مع وسائد وأضواء. إذا شعرت بالقلق، ابتعدي قليلا أو تقدمي لسببًا لنقل المكان. يجب أن يحترم شريكك رغباتك ويشعرك بالراحة.',
 'sequence_length': 969,
 'text': '<|user|>\nيوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى ا

# Using SFTTrainer evaluate

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
from datasets import load_dataset, Dataset
import pandas as pd
import os
from tqdm import tqdm
from evaluate import load

# Set CUDA device
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

from evaluate import load


# Load metrics
rouge = load('rouge')
bleu = load('bleu')
meteor = load('meteor')
bertscore = load('bertscore')

MAX_LEN = 2048
BATCH_SIZE=8
# torch_dtype = torch.float32
torch_dtype = torch.bfloat16

model_path = "BounharAbdelaziz/Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v2"
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


# Load dataset
eval_dataset = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered", split='validation')
# eval_dataset = eval_dataset.select(range(10))

# Summarize dataset

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path).eval()#, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer.padding_side='left'
# Set reasonable default for models without max length
tokenizer.model_max_length = MAX_LEN

# Set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
# Set chat template
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
    
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed. 
    Taken from https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/22
    """
    
    # Handle tuple logits (happens when the model is trained using LoRA)
    if isinstance(logits, tuple):
        logits = logits[1]          # logits[0] is the loss value and logits[1] are the logits used to compute loss
                                    # logits: (tensor(2.0426, device='cuda:0'), tensor([[[ 7.8750,  5.3750,  7.0938,  ..., -4.2500, -4.2500, -4.2500],
                                    #          [ 5.0938,  5.0625,  7.3750,  ..., -1.5312, -1.5312, -1.5312],
                                    #          [ 2.6562, -0.9609,  0.0728,  ..., -2.0312, -2.0312, -2.0312],
                                    #          ...,
                                    #          [ 4.1562,  1.4375, -3.6250,  ..., -2.1250, -2.1250, -2.1250],
                                    #          [ 3.7344, -1.6641, -3.8125,  ..., -1.9688, -1.9688, -1.9688],
                                    #          [ 8.1875, -1.2344, -1.6094,  ..., -3.0938, -3.0938, -3.0938]]],
                                    #        device='cuda:0'))

    # Proceed with argmax
    pred_ids = torch.argmax(logits, dim=-1)

    return pred_ids

@torch.no_grad()
def compute_metrics_causal_lm(eval_pred, tokenizer):
    """Compute ROUGE and BLEU scores for evaluation."""
    predictions, references = eval_pred

    # Ensure predictions and references are in the correct format (list or numpy)
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(references, torch.Tensor):
        references = references.cpu().numpy()

    # Clip token IDs to the valid range
    vocab_size = tokenizer.vocab_size

    def clip_token_ids(token_ids):
        """Clip token IDs to the valid range [0, vocab_size - 1]."""
        # Ensure token_ids is a list or array and clip each value
        return [min(max(token_id, 0), vocab_size - 1) for token_id in token_ids]

    # Decode predictions and references
    decoded_preds = [
        tokenizer.decode(clip_token_ids(pred), skip_special_tokens=True)
        for pred in predictions
    ]
    decoded_refs = [
        tokenizer.decode(clip_token_ids(ref), skip_special_tokens=True)
        for ref in references
    ]

    # Print decoded examples to inspect issues
    print(f'decoded_preds[0]: {decoded_preds[0]}')
    print(f'decoded_refs[0]: {decoded_refs[0]}')

    # If predictions are empty or contain only special tokens, report an issue
    if not decoded_preds[0] or decoded_preds[0] == tokenizer.pad_token:
        print(f"[WARNING] Empty prediction detected for example: {decoded_preds[0]}")

    # Compute ROUGE scores
    rouge_results = rouge.compute(
        predictions=decoded_preds, 
        references=decoded_refs, 
        use_stemmer=True
    )

    # Compute BLEU score
    bleu_results = bleu.compute(
        predictions=decoded_preds, 
        references=decoded_refs
    )

    # Combine the results into a single dictionary
    metrics = {key: rouge_results[key] * 100 for key in ["rouge1", "rouge2", "rougeL", "rougeLsum"]}
    metrics["bleu"] = bleu_results["bleu"] * 100
    
    return metrics

def create_conversation(example):
    """
    Transform the dataset into a conversational format.
    The user provides the text, and the assistant provides the summary.
    """
    # Create a conversation with user and assistant roles
    messages = [
        {"role": "user", "content": example["text"]},  # User provides the text
    ]
    # Return the conversation as a dictionary
    return {"messages": messages}

def apply_chat_template(example, tokenizer):
    """ Apply the chat template to the dataset. """
    example["text"] = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    return example

# Training arguments
training_args = TrainingArguments(
    output_dir="./o",
    per_device_eval_batch_size=BATCH_SIZE,
    # bf16=True,
    # fp16_full_eval=True,
    gradient_checkpointing=True,
)
# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    compute_metrics=lambda x : compute_metrics_causal_lm(x, tokenizer),
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

# Transform the dataset into a conversational format
eval_dataset = eval_dataset.map(create_conversation, remove_columns=["text", "summary"])

eval_dataset = eval_dataset.map(
    apply_chat_template,
    num_proc=os.cpu_count(),
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["messages"],
    desc="Applying chat template..."
)

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/infres/abounhar/.local/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the

In [2]:
eval_dataset[0]

{'summary_model_name': 'Qwen/Qwen2.5-7B-Instruct-AWQ',
 'tokenizer_name': 'Qwen/Qwen2.5-7B-Instruct-AWQ',
 'dataset_source': 'Abdelkareem/wikihow-arabic-summarization',
 'sequence_length': 969,
 'text': '<|user|>\nيوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك.  إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قر

In [3]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 437/437 [00:01<00:00, 296.04 examples/s]


In [4]:
eval_dataset

Dataset({
    features: ['summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'text', 'input_ids', 'attention_mask'],
    num_rows: 437
})

In [5]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<BackBackBackBack<<UnderUnder<UnderUnder<BackUnderBackBackBackBackBackUnderUnderUnderUnder<Under<Back<<BackUnderUnderUnderBackBackBackUnderUnderUnderBackBackBackUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderBackUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderInternetUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder authorisedUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnde

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.5254828929901123, 'eval_model_preparation_time': 0.0047, 'eval_rouge1': 31.289077212390936, 'eval_rouge2': 7.000054785325904, 'eval_rougeL': 30.247449942567627, 'eval_rougeLsum': 30.559380081566523, 'eval_bleu': 2.8362468634132427, 'eval_runtime': 119.0566, 'eval_samples_per_second': 3.671, 'eval_steps_per_second': 0.462}


In [12]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: <<<<<<<<<<<<<<<<<<<<<<<<<<UnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnder<<UnderUnderUnder<UnderUnderUnderUnder<UnderUnderUnderUnder<<<<<<<<<<<<<<<<<<Under<<<UnderUnder<<<Under<<<<<<<<<<<Under<Under<<<<<Under<<Under<<<<<<UnderUnder<<<<<UnderUnderUnder<<<Under<<<<UnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderAndroid<<UnderUnderUnder<UnderUnderUnderUnderUnderUnder<UnderUnder<UnderUnder<UnderUnderUnderUnderUnd

In [6]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: Once<<<Once<<<<<<<<<<<<UnderUnderAuthorUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnder<UnderUnderUnder<Under<UnderUnderAuthorUnderUnderUnderAuthorUnderUnder<<UnderUnder<UnderUnder<<<<<<Under<Under<<<<<<<Under<<Under<<<UnderUnderUnderUnderAuthorUnder<<<<UnderUnderAuthor<<Under<<Under<UnderUnderUnderUnderUnderUnderUnderAndroid<<UnderAuthorAuthorAuthor<UnderUnderUnder<<UnderUnderAuthorUnderUnderUnderUnderUnderUnderUnderUnderUn

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.425283432006836, 'eval_model_preparation_time': 0.0047, 'eval_rouge1': 38.30986197413715, 'eval_rouge2': 17.798467716475482, 'eval_rougeL': 37.15017882326543, 'eval_rougeLsum': 37.75851161995967, 'eval_bleu': 3.9503918932153574, 'eval_runtime': 41.1489, 'eval_samples_per_second': 10.62, 'eval_steps_per_second': 1.337}


In [7]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: |user|>
يجب العديد من الأاليب التي تطلبى حدحدودسة الت لوف م مة من شخصطرف الآخر، العلاقات،رمة،  أناء علاقةقة والواصل بين العلاقات إذا خلال تحدث معماع إلىتحدث فيتبادلترام الآكاره..اق تروج للعا فيمتاع ببعبة بعضكما البعض.  كنتاولتعك أنظهبار بك ليس أمرريقة الوحيدة ل ت أن مقرب،، ف يين إلى توقفير في إذا كان هذه هي س التي تريدينك.ا.  المهمتع أن تشعرب لل شريكك فيكونين بعملات مختلفة الجنسنسية، يمكنك هذاعب علىسي العبية أوريلة جائعة لظه النفسريكك وخفيف الضوترك. أنة لمارسة الجنس. يمكنكحرص في علىجانبه شكما البعض ومجد إلى أكسائد أوقمق و بعضفةة أو إذا أيضا أيضا أن متب التيطبعوب على إذاساعد هذادركنسسكما على نوعجة التاحةتكما إذا إذا كنت تادبي أو قج العلاقةور، فحاولولي أن تجلسقيدي عنليلا عن إذامالوس بعالا ب الأرضاكة و تقلسكدميكما.. إذا أيضا أنضاءة كتاب الكصص الممانسية أوكيضكما البعض أجل تجت اانتباه عنا عن الجنس.مثلغمما تتب تشعرمن بعض بعض بعض رغك الصةص). إذاجلسا بعا علىنا ل بعضصوما يمكنكجلسا عن أات جغنية التيحاولعلسي معلغ من شريكك.اني بمشاهدةكلوكما. يمكنك ا تقملي عن من ذلكاههة الأ كلمك..

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.4264333248138428, 'eval_model_preparation_time': 0.0055, 'eval_rouge1': 26.59967449402053, 'eval_rouge2': 11.045408287198025, 'eval_rougeL': 25.71808991230977, 'eval_rougeLsum': 26.235493743069043, 'eval_bleu': 3.2501948192649843, 'eval_runtime': 86.764, 'eval_samples_per_second': 5.037, 'eval_steps_per_second': 0.634}


In [11]:
trainer.model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [6]:
metrics

{'rouge1': 10.158730158730158,
 'rouge2': 3.333333333333334,
 'rougeL': 10.158730158730158,
 'rougeLsum': 10.158730158730158,
 'bleu': 1.1900292626369438}